In [13]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

In [14]:
# Load training data CSV using Pandas
filename = 'train.csv'
data = pd.read_csv(filename, header=0)
# Separate training data into Y and X
array = data.values
Y = array[:, 1]
X = array[:, 2:18]


In [15]:
def calculateError(y_real, y_predicted):
    error = 0
    for i in range(1, len(y_real)):
        if (y_real[i] != y_predicted[i]):
            error += 1
            
    return error

In [16]:
def crossValidation(X, Y, k, model):
    kf = KFold(k, shuffle=False)
    err = [None] * k
    i = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        model.fit(X_train, y_train)
        y_eval = model.predict(X_test)
        err[i] = calculateError(y_test, y_eval)
        i += 1
    return err


In [20]:
#penalties= ['None', 'l1', 'l2']
solvers = ['lbfgs']
#solvers = ['lbfgs', 'sgd', 'adam']

activationfunctions = ['identity', 'logistic', 'tanh', 'relu']
r = 0.0001 * np.arange(1, 1000, 100)
lambdas =  [*r]
bestError = float("inf")
bestP = 'None'
bestL = 0
bestModel = 0
print("Started")
for p in solvers: 
#for p in penalties: 
    for l in lambdas: 
        for a in activationfunctions: 
        #model = sk.linear_model.Perceptron(penalty=p, alpha=l, fit_intercept=True, max_iter=None, tol=None, shuffle=True, verbose=0, eta0=1.0, n_jobs=1, random_state=0, class_weight=None, warm_start=False, n_iter=None)
        #Test other stuff here
            model = MLPClassifier(solver=p, alpha=l, activation=a, hidden_layer_sizes=(40, 20), random_state=2)
            newError = np.sum(crossValidation(X,Y,10, model))
            if (newError < bestError): 
                bestError = newError
                bestL = l
                bestP = p
                bestModel = model

#Best error for Perceptron: 634. L1 penalty and lambda = 4.01*10^-5
#Hard baseline: MLPClassifier,  lbfgs solver, lambda 0.0801, error of 243, 40,20 hidden layer
print("Best penalty")
print(bestP)
print("Best Lambda")
print(bestL)

print("Best error") 
print(bestError)

Started
Best penalty
lbfgs
Best Lambda
0.0801
Best error
243


In [21]:
#Testing
# Load training data CSV using Pandas
filename = 'test.csv'
data = pd.read_csv(filename, header=0)
# Separate training data into Y and X
array = data.values
X = array[:, 1:17]
print("Predicting")
Y_PRED = bestModel.predict(X)
print("Done)")

# Write prediction to output file
filename = 'prediction.csv'
result = pd.DataFrame(Y_PRED)
result.index = result.index + 2000
result.to_csv(filename, index_label='Id', header=['y'])

Predicting
Done)
